In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')
WANDB_KEY = userdata.get('WANDB_KEY')

In [ ]:
!pip install -U accelerate transformers datasets peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.8/410.8 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip uninstall bitsandbytes -y
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.4 MB/s eta 0:00:00


In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login(key=WANDB_KEY)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: inclinedadarsh (inclinedadarsh-kk-wagh-institute) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset, DatasetDict
from peft import LoraConfig, get_peft_model
import torch

In [ ]:
# Load the model
dataset = load_dataset('databricks/databricks-dolly-15k')

# Filter rows only with open_qa or general_qa category
dataset = dataset.filter(lambda example: example['category'] == 'open_qa'  or example['category'] == 'general_qa')

# Select only first 100 rows for faster training
dataset = DatasetDict({
    'train': dataset['train'].select(range(100))
})

README.md:   0%|          | 0.00/8.20k [00:00<?, ?B/s]

databricks-dolly-15k.jsonl:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

Filter:   0%|          | 0/15011 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 100
    })
})

In [ ]:
def format_example(example):
    text = (
        f"<start_of_turn>user\n",
        f"{example['instruction']}<end_of_turn>\n",
        f"<start_of_turn>assistant\n",
        f"{example['response']}<end_of_turn>"
    )

    return {'text': "".join(text)}

dataset = dataset.map(format_example)

print(dataset['train'][0]['text'])

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

<start_of_turn>user
Why can camels survive for long without water?<end_of_turn>
<start_of_turn>assistant
Camels use the fat in their humps to keep them filled with energy and hydration for long periods of time.<end_of_turn>


In [ ]:
model_name = 'google/gemma-3-1b-it'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_TOKEN)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [ ]:
def tokenize_example(example):
    tokens = tokenizer(
        example['text'],
        padding='max_length',
        truncation=True,
        max_length=512,
    )

    tokens['labels'] = tokens['input_ids'].copy()

    return tokens

tokenized_dataset = dataset.map(tokenize_example, batched=True, remove_columns=['text', 'instruction', 'context', 'response', 'category'])

print(tokenized_dataset)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"":0},
    attn_implementation='eager',
    token=HF_TOKEN
)

In [ ]:
for name, param in model.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

model.embed_tokens.weight: requires_grad=True
model.layers.0.self_attn.q_proj.weight: requires_grad=False
model.layers.0.self_attn.k_proj.weight: requires_grad=False
model.layers.0.self_attn.v_proj.weight: requires_grad=False
model.layers.0.self_attn.o_proj.weight: requires_grad=False
model.layers.0.self_attn.q_norm.weight: requires_grad=True
model.layers.0.self_attn.k_norm.weight: requires_grad=True
model.layers.0.mlp.gate_proj.weight: requires_grad=False
model.layers.0.mlp.up_proj.weight: requires_grad=False
model.layers.0.mlp.down_proj.weight: requires_grad=False
model.layers.0.input_layernorm.weight: requires_grad=True
model.layers.0.post_attention_layernorm.weight: requires_grad=True
model.layers.0.pre_feedforward_layernorm.weight: requires_grad=True
model.layers.0.post_feedforward_layernorm.weight: requires_grad=True
model.layers.1.self_attn.q_proj.weight: requires_grad=False
model.layers.1.self_attn.k_proj.weight: requires_grad=False
model.layers.1.self_attn.v_proj.weight: requi

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total Parameters: {total_params / 1e9:.2f}B")
print(f"Trainable Parameters (Full Fine-Tuning): {trainable_params / 1e9:.2f}B")

Total Parameters: 1.00B
Trainable Parameters (Full Fine-Tuning): 1.00B


In [ ]:
lora_config = LoraConfig(
    r=4,
    lora_alpha=4,
    target_modules=['q_proj', 'v_proj'],
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM'
)

In [ ]:
lora_model = get_peft_model(model, lora_config)

In [ ]:
lora_model.print_trainable_parameters()

trainable params: 372,736 || all params: 1,000,258,688 || trainable%: 0.0373


In [ ]:
lora_model.gradient_checkpointing_enable()

In [ ]:
training_args = TrainingArguments(
    output_dir='./gemma-2-lora-finetuned',
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    bf16=True,
    optim='adamw_bnb_8bit',
    logging_steps=2,
    report_to='wandb',
    save_strategy='epoch',
    remove_unused_columns=False  # Important for LoRA
)

In [ ]:
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_dataset['train']
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
wandb.init(project="gemma-2-finetuning", name="gemma-2b-lora-run1")

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn